In [3]:
import csv

# import cv2
import os
# import tensorflow as tf
# import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.layers import Dense, Flatten, Conv2D
# from tensorflow.keras import Model
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# python standard libraries
# import os
# from os import path

import random
import fnmatch
import datetime
import pickle

# data processing
import numpy as np
np.set_printoptions(formatter={'float_kind':lambda x: "%.4f" % x})



import pandas as pd
pd.set_option('display.width', 300)
pd.set_option('display.float_format', '{:,.4f}'.format)
pd.set_option('display.max_colwidth', 200)

# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
print( f'tf.__version__: {tf.__version__}' )
print( f'keras.__version__: {keras.__version__}' )

# sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# imaging
import cv2
from imgaug import augmenters as img_aug
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from PIL import Image, ImageEnhance


tf.__version__: 2.4.1
keras.__version__: 2.4.0


In [5]:
y = []
x_names = []

with open('/Users/Kamyab/Documents/UoN/MLiS_2/Project/training_norm.csv', mode='r') as infile:
    reader = csv.reader(infile)
    next(reader)
#     print(reader)
    with open('training_norm_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
#         c = 0
        for rows in reader:
#             if c > 100:
#                 break
            f = rows[0]
            a = float(rows[1])
            s = float(rows[2])
            y.append([a,s])
            filename = str(f) + '.png'
            x_names.append(filename)
            

In [6]:
print(np.shape(x_names))

(13796,)


In [7]:
print(np.shape(y))

(13796, 2)


In [8]:
def image_data_generator(image_paths, steering_angles, batch_size, is_training):
    while True:
        batch_images = []
        batch_steering_angles = []
        path = '/Users/Kamyab/Documents/UoN/MLiS_2/Project/Data/training_data/training_data'
       
        for i in range(batch_size):
            random_index = random.randint(0, len(image_paths) - 1)
           
            image_path = path + image_paths[random_index]
            image = cv2.imread(image_path)
            steering_angle = steering_angles[random_index]
            if is_training:
                # training: augment image
                image, steering_angle = random_augment(image, steering_angle)
             
            image = img_preprocess(image)
            batch_images.append(image)
            batch_steering_angles.append(steering_angle)
           
        yield( np.asarray(batch_images), np.asarray(batch_steering_angles))

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split( x_names, y, test_size=0.2)
print("Training data: %d\nValidation data: %d" % (len(X_train), len(X_valid)))

Training data: 11036
Validation data: 2760


In [23]:
def zoom(image):
    zoom = img_aug.Affine(scale=(1, 1.3))  # zoom from 100% (no zoom) to 130%
    image = zoom.augment_image(image)
    return image

In [24]:
def pan(image):
    # pan left / right / up / down about 10%
    pan = img_aug.Affine(translate_percent= {"x" : (-0.1, 0.1), "y": (-0.1, 0.1)})
    image = pan.augment_image(image)
    return image

In [25]:
def adjust_brightness(image):
    # increase or decrease brightness by 30%
    brightness = img_aug.Multiply((0.7, 1.3))
    image = brightness.augment_image(image)
    return image

In [26]:
def blur(image):
    kernel_size = random.randint(1, 5)  # kernel larger than 5 would make the image way too blurry
    image = cv2.blur(image,(kernel_size, kernel_size))
   
    return image

In [27]:
def random_flip(image, steering_angle):
    is_flip = random.randint(0, 1)
    if is_flip == 1:
        # randomly flip horizon
        image = cv2.flip(image,1)
        steering_angle[0] = (80 - steering_angle[0]*80)/80
        #(180 - (steering_angle[0]*80 + 50) -50)/80
   
    return image, steering_angle


In [28]:
def random_augment(image, steering_angle):
    if np.random.rand() < 0.5:
        image = pan(image)
    if np.random.rand() < 0.5:
        image = zoom(image)
    if np.random.rand() < 0.5:
        image = blur(image)
    if np.random.rand() < 0.5:
        image = adjust_brightness(image)
    image, steering_angle = random_flip(image, steering_angle)
    
    return image, steering_angle



# show a few randomly augmented images
# ncol = 2
# nrow = 10
# fig, axes = plt.subplots(nrow, ncol, figsize=(15, 50))

# for i in range(nrow):
#     rand_index = random.randint(0, len(image_paths) - 1)
#     image_path = image_paths[rand_index]
#     steering_angle_orig = steering_angles[rand_index]
    
#     image_orig = my_imread(image_path)
#     image_aug, steering_angle_aug = random_augment(image_orig, steering_angle_orig)
    
#     axes[i][0].imshow(image_orig)
#     axes[i][0].set_title("original, angle=%s" % steering_angle_orig)
#     axes[i][1].imshow(image_aug)
#     axes[i][1].set_title("augmented, angle=%s" % steering_angle_aug)
    

In [29]:
def img_preprocess(image):
    height, _, _ = image.shape
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (160,120)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing
    return image

In [27]:
# def nvidia_model():
#     model = Sequential(name='Nvidia_Model')
    
#     # elu=Expenential Linear Unit, similar to leaky Relu
#     # skipping 1st hiddel layer (nomralization layer), as we have normalized the data
    
#     # Convolution Layers
#     model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu')) 
#     model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu')) 
#     model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu')) 
#     model.add(Conv2D(64, (3, 3), activation='elu')) 
#     model.add(Dropout(0.2)) # not in original model. added for more robustness
#     model.add(Conv2D(64, (3, 3), activation='elu')) 
    
#     # Fully Connected Layers
#     model.add(Flatten())
#     model.add(Dropout(0.2)) # not in original model. added for more robustness
#     model.add(Dense(100, activation='elu'))
#     model.add(Dense(50, activation='elu'))
#     model.add(Dense(10, activation='elu'))
#     model.add(Dense(2, activation='tanh')) 
#     model.add(Dense(2, activation='sigmoid')) 

    
#     # output layer: turn angle (from 45-135, 90 is straight, <90 turn left, >90 turn right)
#     model.add(Dense(1))
#     model.add(Dense(2))
    
#     # since this is a regression problem not classification problem,
#     # we use MSE (Mean Squared Error) as loss function
#     optimizer = Adam(lr=1e-3) # lr is learning rate
#     model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    
#     return model

# model = nvidia_model()
# print(model.summary())

In [10]:
import pydot
from keras_visualizer import visualizer 
# from keras.utils.vis_utils import plot_model

def nvidia_model():
    model = Sequential(name='Nvidia_Model')
    
    # elu=Expenential Linear Unit, similar to leaky Relu
    # skipping 1st hiddel layer (nomralization layer), as we have normalized the data
    
    # Convolution Layers
    model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(120, 160, 3), activation='elu')) 
    model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    
    # output layer: turn angle (from 45-135, 90 is straight, <90 turn left, >90 turn right)
    model.add(Dense(2, activation='sigmoid'))#, activation='tanh')) 
    
    # since this is a regression problem not classification problem,
    # we use MSE (Mean Squared Error) as loss function
    optimizer = Adam(lr=1e-3) # lr is learning rate
    model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
 

 

    return model

 

model = nvidia_model()
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
visualizer(model, format='png', view=True)
print(model.summary())

ValueError: Keras Visualizer: Error while visualizing

In [6]:
from ann_visualizer.visualize import ann_viz;

ann_viz(model, view=True, title= 'MyNeural Network')

ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tpdf', '-O', 'network.gv'], make sure the Graphviz executables are on your systems' PATH

In [31]:
# saves the model weights after each epoch if the validation loss decreased
model_output_dir = '/Users/Kamyab/Documents/'

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(model_output_dir,'lane_navigation_check_trial2.h5'), verbose=1, save_best_only=True)

history = model.fit_generator(image_data_generator( X_train, y_train, batch_size=100, is_training=True),
                              steps_per_epoch=300,
                              epochs=100,
                              validation_data = image_data_generator( X_valid, y_valid, batch_size=100, is_training=False),
                              validation_steps=20, 
                              verbose=1,
                              shuffle=1,
                              callbacks=[checkpoint_callback])
# always save model output as soon as model finishes training
model.save(os.path.join(model_output_dir,'lane_navigation_final_trial2.h5'))

TypeError: %d format: a number is required, not str

In [36]:
model = load_model(os.path.join(model_output_dir,'lane_navigation_final.h5'))
       
def compute_steering_angle(self, frame):
    preprocessed = img_preprocess(frame)
    X = np.asarray([preprocessed])
    steering_angle = model.predict(X)[0]
    return steering_angle

In [22]:
path_file = '/Users/Kamyab/Documents/UoN/MLiS_2/Project/Data/test_data/test_data/'
test_names = np.arange(1020) + 1
with open(path_file + 'submission2.csv', mode='w', newline='') as submission:
    writer = csv.writer(submission, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['image_id', 'angle', 'speed'])
    for name in test_names:     
        img = cv2.imread(path_file + str(name) + '.png')
#         print(img)
        prediction = compute_steering_angle(model, img)
        if prediction[1] < 0.5:
            prediction[1] = 0
        else:
            prediction[1] = 1
        writer.writerow([name, prediction[0], prediction[1]])
        print(name, prediction[0], prediction[1])
   
 

1 0.54785573 0.0
2 0.5234339 1.0
3 0.5076484 0.0
4 0.4993941 1.0
5 0.5032775 0.0
6 0.52875423 1.0
7 0.5275266 1.0
8 0.49623847 1.0
9 0.5327319 0.0
10 0.50624985 1.0
11 0.5449625 0.0
12 0.5184102 0.0
13 0.5139861 1.0
14 0.55536133 0.0
15 0.51239425 1.0
16 0.50592756 1.0
17 0.5602645 0.0
18 0.564807 0.0
19 0.5331507 0.0
20 0.5036785 1.0
21 0.5055464 1.0
22 0.5208119 1.0
23 0.5031266 1.0
24 0.5721595 0.0
25 0.52658737 0.0
26 0.54369473 1.0
27 0.5502027 0.0
28 0.5436015 0.0
29 0.53282213 0.0
30 0.52473307 1.0
31 0.5043597 1.0
32 0.5125564 1.0
33 0.53761387 0.0
34 0.5210462 1.0
35 0.52362335 0.0
36 0.514337 1.0
37 0.51646346 1.0
38 0.5187133 1.0
39 0.53245854 0.0
40 0.5274891 1.0
41 0.5203018 1.0
42 0.51845163 0.0
43 0.5075086 1.0
44 0.5332666 0.0
45 0.5126204 1.0
46 0.52813494 0.0
47 0.5415854 0.0
48 0.5160197 0.0
49 0.5008421 1.0
50 0.5141042 1.0
51 0.51297086 1.0
52 0.5203105 1.0
53 0.5505859 0.0
54 0.5245192 1.0
55 0.5535634 0.0
56 0.5405551 0.0
57 0.5433821 0.0
58 0.52197164 0.0
59 0.5